<a href="https://colab.research.google.com/github/VarshithaNuligonda/CredResolve/blob/main/voice_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q groq langchain-groq langgraph gTTS openai-whisper
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [ ]:
import os
os.environ["GROQ_API_KEY"] = "apikey"#create your own apikey


In [ ]:
from IPython.display import HTML, display,Audio
from google.colab import output
import base64

RECORD_JS = """
const record = async (time) => {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const recorder = new MediaRecorder(stream);
  const chunks = [];
  recorder.ondataavailable = e => chunks.push(e.data);
  recorder.start();

  await new Promise(resolve => setTimeout(resolve, time));
  recorder.stop();

  return new Promise(resolve => {
    recorder.onstop = async () => {
      const blob = new Blob(chunks);
      const reader = new FileReader();
      reader.readAsDataURL(blob);
      reader.onloadend = () => resolve(reader.result);
    };
  });
};
"""

def get_audio_from_browser(seconds=6):
    display(HTML(f"<script>{RECORD_JS}</script>"))
    audio_b64 = output.eval_js(f"record({seconds*1000})")
    audio_bytes = base64.b64decode(audio_b64.split(',')[1])
    with open("user_audio.wav", "wb") as f:
        f.write(audio_bytes)
    return "user_audio.wav"


In [ ]:
def get_audio():
    print("🎤 Recording... Please speak now")
    audio_path = get_audio_from_browser(seconds=6)  # 6-second recording
    print("✅ Recording finished")
    return audio_path


In [ ]:
import whisper

stt_model = whisper.load_model("base")

def speech_to_text(audio_file, lang_code):
    result = stt_model.transcribe(
        audio_file,
        language=lang_code,
        task="transcribe"
    )
    return result["text"].strip()


In [ ]:
def eligibility_tool(age, income):
    if age is None or income is None:
        return {"status": "missing"}
    if age >= 18 and income <= 250000:
        return {"status": "eligible"}
    return {"status": "not_eligible"}


In [ ]:
def scheme_tool():
    return {
        "PM_KISAN": {"type": "farmer"},
        "AYUSHMAN_BHARAT": {"type": "health"},
        "NSAP": {"type": "social"}
    }


In [ ]:
from typing import TypedDict, List

class WelfareState(TypedDict):
    query: str
    language: str
    history: List[str]
    age: int | None
    income: int | None
    tool_data: dict
    response: str


In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)


In [ ]:
def planner(state):
    # Ask for missing info
    if state["age"] is None:
        return {"response": "మీ వయస్సు ఎంత? సంఖ్యలో చెప్పండి.", "awaiting_user": True}
    if state["income"] is None:
        return {"response": "మీ వార్షిక ఆదాయం ఎంత? సంఖ్యలో చెప్పండి.", "awaiting_user": True}

    # Use tools
    eligibility = eligibility_tool(state["age"], state["income"])
    schemes = scheme_tool()  # List or dict of schemes

    # LLM prompt: include user query, eligibility, schemes
    prompt = f"""
వినియోగదారుడు అడిగినది: "{state['query']}"
వయస్సు: {state['age']}
ఆదాయం: {state['income']}

ప్రశ్నకు సరైన సమాధానం ఇవ్వండి:
1. వినియోగదారుడు ఏ ప్రభుత్వ పథకాల కోసం అర్హత కలిగి ఉన్నాడో చెప్పండి
2. ప్రతి పథకం చిన్న వివరణ ఇవ్వండి
3. సమాధానం పూర్తిగా {state['language']} లో ఉండాలి

లభ్యమయ్యే పథకాలు: {schemes}
అర్హత: {eligibility}
"""

    res = llm.invoke(prompt)
    return {"response": res.content, "awaiting_user": False}


In [ ]:
from langgraph.graph import StateGraph, END

builder = StateGraph(WelfareState)
builder.add_node("planner", planner)
builder.set_entry_point("planner")
builder.add_edge("planner", END)

agent = builder.compile()


In [ ]:
import re

def extract_age_income(text):
    # Remove non-digit characters except dot
    cleaned = re.sub(r"[^\d.]", " ", text)
    nums = [float(x) for x in cleaned.split() if x.strip().isdigit() or x.replace(".","",1).isdigit()]

    age = nums[0] if len(nums) >= 1 else None
    income = nums[1] if len(nums) >= 2 else None
    return age, income


In [ ]:
from gtts import gTTS
from IPython.display import Audio, display

def speak(text, lang_code):
    tts = gTTS(text=text, lang=lang_code)
    tts.save("reply.mp3")
    display(Audio("reply.mp3", autoplay=True))


In [ ]:
# 🌐 LANGUAGE CONFIG
LANGUAGES = {
    "1": {"name": "Telugu",  "code": "te", "prompt": "తెలుగు"},
    "2": {"name": "Marathi","code": "mr", "prompt": "मराठी"},
    "3": {"name": "Tamil",  "code": "ta", "prompt": "தமிழ்"},
    "4": {"name": "Hindi",  "code": "hi", "prompt": "हिंदी"},
    "5": {"name": "Bengali","code": "bn", "prompt": "বাংলা"},
    "6": {"name": "Odia",   "code": "or", "prompt": "ଓଡ଼ିଆ"},
}

# 🗣️ ASK USER TO CHOOSE LANGUAGE
print("Select Language:")
for k, v in LANGUAGES.items():
    print(f"{k}. {v['name']}")

choice = input("Enter choice: ").strip()

# ✅ SAFE DEFAULT (Hindi if invalid)
lang = LANGUAGES.get(choice, LANGUAGES["4"])

print(f"Language selected: {lang['name']}")
conversation_memory = []
age = None
income = None

def get_number_from_user(prompt_text, lang_code):
    """Try to get a numeric value from voice first, fallback to text input."""
    while True:
        # 1️⃣ Try voice input
        print(f"🎤 {prompt_text} (You can speak now)")
        audio_file = get_audio()
        spoken_text = speech_to_text(audio_file, lang_code)
        print("🎧 Whisper recognized:", spoken_text)

        # Extract numbers robustly
        cleaned = re.sub(r"[^\d.]", " ", spoken_text)
        nums = [x for x in cleaned.split() if x.replace(".","",1).isdigit()]
        if nums:
            print(f"✅ Recognized number: {nums[0]}")
            return float(nums[0])

        # 2️⃣ Fallback to typed input
        typed = input(f"⚠️ Couldn't understand. Please type {prompt_text}: ").strip()
        if typed.replace(".","",1).isdigit():
            return float(typed)
        else:
            print("⚠️ Invalid input. Please enter digits only.")

while True:
    # --- Step 1: Capture user query via voice ---
    audio = get_audio()
    user_text = speech_to_text(audio, lang["code"])
    print("🎧 User said:", user_text)
    conversation_memory.append(f"User: {user_text}")

    # --- Step 2: Get age if missing ---
    if age is None:
        age = get_number_from_user("మీ వయస్సు (Age) చెప్పండి", lang["code"])

    # --- Step 3: Get income if missing ---
    if income is None:
        income = get_number_from_user("మీ వార్షిక ఆదాయం (Income) చెప్పండి", lang["code"])

    # --- Step 4: Build state for agent ---
    state = {
        "query": user_text,
        "language": lang["prompt"],
        "history": conversation_memory,
        "age": age,
        "income": income
    }

    # --- Step 5: Invoke agent ---
    result = agent.invoke(state)
    print("Agent:", result["response"])
    speak(result["response"], lang["code"])
    conversation_memory.append(f"Agent: {result['response']}")

    # --- Step 6: Stop loop after final recommendation ---
    if not result.get("awaiting_user", True):
        print("✅ Final recommendation given. Exiting.")
        break


Select Language:
1. Telugu
2. Marathi
3. Tamil
4. Hindi
5. Bengali
6. Odia
Enter choice: 1
Language selected: Telugu
🎤 Recording... Please speak now


✅ Recording finished


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🎧 User said: dm różne
🎤 మీ వయస్సు (Age) చెప్పండి (You can speak now)
🎤 Recording... Please speak now


✅ Recording finished


...

/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🎧 Whisper recognized: India
⚠️ Couldn't understand. Please type మీ వార్షిక ఆదాయం (Income) చెప్పండి: 20000
Agent: వినియోగదారుడు అర్హత కలిగి ఉన్న ప్రభుత్వ పథకాలు:

1. పిఎం కిసాన్ (PM_KISAN): ఈ పథకం రైతులకు ఆర్థిక సహాయం అందించడానికి ఉద్దేశించినది. రైతులు తమ పొలాలలో పంటలు పండించడానికి అవసరమైన ఆర్థిక సహాయం ఈ పథకం ద్వారా అందించబడుతుంది.
2. ఆయుష్మాన్ భారత్ (AYUSHMAN_BHARAT): ఈ పథకం ఆరోగ్య సంరక్షణకు సంబంధించినది. ఈ పథకం ద్వారా వినియోగదారులు ఆరోగ్య సంరక్షణ సేవలను సులభంగా పొందవచ్చు.
3. ఎన్ఎస్ఏపి (NSAP): ఈ పథకం సామాజిక సంక్షేమానికి సంబంధించినది. ఈ పథకం ద్వారా వినియోగదారులు సామాజిక సంక్షేమ పథకాల ద్వారా ప్రయోజనం పొందవచ్చు.

ఈ పథకాలకు వినియోగదారుడు అర్హత కలిగి ఉన్నాడు. కాబట్టి, వినియోగదారుడు ఈ పథకాల ద్వారా ప్రయోజనం పొందవచ్చు.


🎤 Recording... Please speak now


✅ Recording finished


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
